In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
Data= pd.read_csv('https://raw.githubusercontent.com/Abeyaantrix2Internship/intern/main/50_Startups.csv')
Data.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [3]:
Data.shape

(50, 5)

In [4]:
Data.describe()

,R&D Spend,Administration,Marketing Spend,Profit
count,50.000000,50.000000,50.000000,50.000000
mean,73721.615600,121344.639600,211025.097800,112012.639200
std,45902.256482,28017.802755,122290.310726,40306.180338
min,0.000000,51283.140000,0.000000,14681.400000
25%,39936.370000,103730.875000,129300.132500,90138.902500
50%,73051.080000,122699.795000,212716.240000,107978.190000
75%,101602.800000,144842.180000,299469.085000,139765.977500
max,165349.200000,182645.560000,471784.100000,192261.830000


In [5]:
Data.describe(include='all')

,R&D Spend,Administration,Marketing Spend,State,Profit
count,50.000000,50.000000,50.000000,50,50.000000
unique,NaN,NaN,NaN,3,NaN
top,NaN,NaN,NaN,New York,NaN
freq,NaN,NaN,NaN,17,NaN
mean,73721.615600,121344.639600,211025.097800,NaN,112012.639200
std,45902.256482,28017.802755,122290.310726,NaN,40306.180338
min,0.000000,51283.140000,0.000000,NaN,14681.400000
25%,39936.370000,103730.875000,129300.132500,NaN,90138.902500
50%,73051.080000,122699.795000,212716.240000,NaN,107978.190000
75%,101602.800000,144842.180000,299469.085000,NaN,139765.977500


In [6]:
Data.isnull().sum()

R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

In [7]:
#EDA
#split data into dependent and independent variables
Y= Data.iloc[:,-1:]# dependent variables
X= Data.iloc[:,[0,1,2,3]]# independent variables

In [8]:
print(X.State.unique())
print('No of columns in Fuel type Column:',len(X.State.unique()))

['New York' 'California' 'Florida']
No of columns in Fuel type Column: 3


In [9]:
print(X.shape)
print(Y.shape)

(50, 4)
(50, 1)


In [10]:
X.head()

,R&D Spend,Administration,Marketing Spend,State
0,165349.20,136897.80,471784.10,New York
1,162597.70,151377.59,443898.53,California
2,153441.51,101145.55,407934.54,Florida
3,144372.41,118671.85,383199.62,New York
4,142107.34,91391.77,366168.42,Florida


In [11]:
# now apply Data preprocessing on State column
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
CT= ColumnTransformer([('encoder',OneHotEncoder(),[3])], remainder='passthrough')
#X_En=CT.fit_transform(X)
X_En=np.array(CT.fit_transform(X), dtype=np.float64)
print(X_En[1:2,:])

[[1.0000000e+00 0.0000000e+00 0.0000000e+00 1.6259770e+05 1.5137759e+05
  4.4389853e+05]]


In [12]:
#removing Dummy column
X_encoded=X_En[:,1:]
print(X_encoded[1:2,:])

[[     0.        0.   162597.7  151377.59 443898.53]]


In [13]:
## Feature engg
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X_encoded,Y, test_size=0.2, random_state=0)

In [14]:
X_train.shape

(40, 5)

In [15]:
# apply linear regression
from sklearn.linear_model import LinearRegression
MLR= LinearRegression()
MLR.fit(X_train,Y_train)

LinearRegression()

In [16]:
print(X_train.shape,Y_train.shape)

(40, 5) (40, 1)


In [17]:
#check on test data
Y_predicted= MLR.predict(X_test)

In [18]:
MLR.coef_

array([[-9.59284160e+02,  6.99369053e+02,  7.73467193e-01,
         3.28845975e-02,  3.66100259e-02]])

In [19]:
MLR.intercept_

array([42554.16761777])

In [20]:
from sklearn.metrics import mean_squared_error
MSE= mean_squared_error(Y_predicted, Y_test)
print(MSE)
print(np.sqrt(MSE))

83502864.03250548
9137.990152791011


In [21]:
from sklearn.metrics import r2_score
R2= r2_score(Y_predicted,Y_test)
print(R2)

0.9293749209318587


In [22]:
import statsmodels.api as ols
X0=np.ones(Y_train.shape)# created array same size as training data
X1=X_train
X_OLS=np.append(arr=X0, values=X1, axis=1)#append training data with X0
print(X_OLS.shape)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


(40, 6)


In [23]:
import statsmodels.api as ols
MLR_OLS = ols.OLS(Y_train, X_OLS).fit()# summary of training data
print(MLR_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                 Profit   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     129.7
Date:                Wed, 09 Feb 2022   Prob (F-statistic):           3.91e-21
Time:                        04:38:16   Log-Likelihood:                -421.10
No. Observations:                  40   AIC:                             854.2
Df Residuals:                      34   BIC:                             864.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.255e+04   8358.538      5.091      0.0

In [24]:
import statsmodels.api as ols
X_BE = X_OLS[:,[0,1,2,3,4,5]]# x0,x1,x2,x3,x4,x5 --> Y=x0b0+x1b1+x2b2+.....x5b5
MLR_OLS = ols.OLS(Y_train, X_BE).fit()# summary of training data
print(MLR_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                 Profit   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     129.7
Date:                Wed, 09 Feb 2022   Prob (F-statistic):           3.91e-21
Time:                        04:38:16   Log-Likelihood:                -421.10
No. Observations:                  40   AIC:                             854.2
Df Residuals:                      34   BIC:                             864.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.255e+04   8358.538      5.091      0.0

In [25]:
# x0=0.0, x1=0.814, x2=0.850(High), x3=0.0, x4=0.624 ,X5=0.068
#according to statistic remove X2 and build the regression model
#remove X2
X_BE = X_OLS[:,[0,1,3,4,5]]# x0,x1,x2,x3,x4,x5 --> Y=x0b0+x1b1+x2b2+.....x5b5

#x0=x0, x1=x1, x2=x3, x3=x4, x4=x5
MLR_OLS = ols.OLS(Y_train, X_BE).fit()# summary of training data
print(MLR_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                 Profit   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     166.7
Date:                Wed, 09 Feb 2022   Prob (F-statistic):           2.87e-22
Time:                        04:38:16   Log-Likelihood:                -421.12
No. Observations:                  40   AIC:                             852.2
Df Residuals:                      35   BIC:                             860.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.292e+04   8020.397      5.352      0.0

In [26]:
# x0=0.0, x1=0.72(high), x2=0.00, x3=0.629, x4=0.065
#according to statistic remove X2 and build the regression model
#remove X1
X_BE = X_OLS[:,[0,3,4,5]]# x0,x1,x2,x3,x4,x5 --> Y=x0b0+x1b1+x2b2+.....x5b5

#x0=x0, x1=x1, x2=x3, x3=x4, x4=x5
#x0=x0, x1=x3, x3(x4)=x5, x3(x4)=x5
MLR_OLS = ols.OLS(Y_train, X_BE).fit()# summary of training data
print(MLR_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                 Profit   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     227.8
Date:                Wed, 09 Feb 2022   Prob (F-statistic):           1.85e-23
Time:                        04:38:16   Log-Likelihood:                -421.19
No. Observations:                  40   AIC:                             850.4
Df Residuals:                      36   BIC:                             857.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.299e+04   7919.773      5.428      0.0

In [27]:
# x0=0.0, x1=0.72(high), x2=0.00, x3=0.629, x4=0.065
#according to statistic remove X2 and build the regression model
#remove X4
X_BE = X_OLS[:,[0,3,5]]# x0,x1,x2,x3,x4,x5 --> Y=x0b0+x1b1+x2b2+.....x5b5

#x0=x0, x1=x1, x2=x3, x3=x4, x4=x5
#x0=x0, x1=x3, x3(x4)=x5, x3(x4)=x5
MLR_OLS = ols.OLS(Y_train, X_BE).fit()# summary of training data
print(MLR_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                 Profit   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.947
Method:                 Least Squares   F-statistic:                     349.0
Date:                Wed, 09 Feb 2022   Prob (F-statistic):           9.65e-25
Time:                        04:38:16   Log-Likelihood:                -421.30
No. Observations:                  40   AIC:                             848.6
Df Residuals:                      37   BIC:                             853.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.635e+04   2971.236     15.598      0.0

In [28]:
X_BE = X_OLS[:,[0,3]]# x0,x1,x2,x3,x4,x5 --> Y=x0b0+x1b1+x2b2+.....x5b5

#x0=x0, x1=x1, x2=x3, x3=x4, x4=x5
#x0=x0, x1=x3, x3(x4)=x5, x3(x4)=x5
MLR_OLS = ols.OLS(Y_train, X_BE).fit()# summary of training data
print(MLR_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                 Profit   R-squared:                       0.945
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     652.4
Date:                Wed, 09 Feb 2022   Prob (F-statistic):           1.56e-25
Time:                        04:38:16   Log-Likelihood:                -423.09
No. Observations:                  40   AIC:                             850.2
Df Residuals:                      38   BIC:                             853.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.842e+04   2842.717     17.032      0.0